In [1]:
import json
import os
import openai
from dotenv import load_dotenv
load_dotenv()

openai.api_key = os.environ["OPENAI_API_KEY"]

In [2]:
selected_model = "gpt-4-1106-preview"
temperature=0.6

In [3]:
from genai.eyfs import (
    TextGenerator,
)
from genai import MessageTemplate, FunctionTemplate

/Users/karlis.kanders/Documents/code/discovery_generative_ai/.venv/lib/python3.9/site-packages/pinecone/index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [6]:
def read_jsonl(path: str) -> list:
    """Read a JSONL file."""
    with open(path, "r") as f:
        return [json.loads(line) for line in f.readlines()]
    
def generate_signals_texts(signals_data: dict, chosen_signals: list = None):
    signals = [signal["short_name"] for signal in signals_data]
    signals_titles = [signal["title"] for signal in signals_data]
    signals_summaries = [signal["summary"] for signal in signals_data]

    if chosen_signals is None:
        chosen_signals = signals

    # Combine titles and summaries into a single string
    signals_description = ""
    for short_name, title, summary in zip(signals, signals_titles, signals_summaries):
        if short_name in chosen_signals:
            signals_description += f"Signal '{short_name}': {title}\n{summary}\n\n"

    return signals_description    

def generate_action_texts(action_data: dict):
    actions = [a["name"] for a in action_data]
    action_descriptions = [a["description"] for a in action_data]
    action_text = ""
    for name, description in zip(actions, action_descriptions):
        action_text += f"Action '{name}': {description}\n\n"
    return action_text    

In [7]:
path_signals_data = "data/signals_2023.json"
signals_data = json.load(open(path_signals_data, "r"))
signals_dict = {s['short_name']: s for s in signals_data}
signals_descriptions = generate_signals_texts(signals_data) 
signals = [s['short_name'] for s in signals_data]

In [8]:
path_actions_data = "data/intent_actions.json"
actions_data = json.load(open(path_actions_data, "r"))
actions_descriptions = generate_action_texts(actions_data)
actions = [a['name'] for a in actions_data]

## Top signal prompt

In [74]:
path_func_top_signal = "data/func_top_signal.json"
path_prompt_top_signal = "data/prompt_top_signal.jsonl"

In [34]:
func_top_signal = json.loads(open(path_func_top_signal).read())
func_top_signal['parameters']['properties']['prediction']['enum'] = signals
func_top_signal

{'name': 'predict_top_signal',
 'description': 'Predict which of the signals are most relevant to user input',
 'parameters': {'type': 'object',
  'properties': {'prediction': {'type': 'string',
    'enum': ['robochefs', 'abundant_energy'],
    'description': 'The predicted most relevant signal'}},
  'required': ['prediction']}}

In [18]:
prompt_top_signal = read_jsonl(path_prompt_top_signal)
prompt_top_signal

[{'role': 'user',
  'content': '###Instructions### Predict which of the following future signals is the most relevant to user input.\n\n###Future signal summaries###\n{signals}\n\n###User input:\n{user_input}'}]

In [68]:
user_input = "I'm living in country side"

In [69]:
message = MessageTemplate.load(path_prompt_top_signal)
function = FunctionTemplate.load(func_top_signal)

In [70]:
response = TextGenerator.generate(
        model=selected_model,
        temperature=temperature,
        messages=[message],
        message_kwargs={"signals": signals_descriptions, "user_input": user_input},
        stream=False,
        functions=[function.to_prompt()],
        function_call={"name": "predict_top_signal"},
)

In [71]:
response['choices'][0]['message']

<OpenAIObject at 0x11b379770> JSON: {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "predict_top_signal",
    "arguments": "{\n\"prediction\": \"abundant_energy\"\n}"
  }
}

In [72]:
answer = json.loads(response['choices'][0]['message']['function_call']['arguments'])
answer

{'prediction': 'abundant_energy'}

## Top 3 signals prompt

In [25]:
path_func_top_signals = "data/func_top_three_signals.json"
path_prompt_top_signals = "data/prompt_top_three_signals.jsonl"

In [26]:
func_top_signals = json.loads(open(path_func_top_signals).read())
func_top_signals['parameters']['properties']['prediction']['items']['enum'] = signals

In [27]:
prompt_top_signals = read_jsonl(path_prompt_top_signals)
prompt_top_signals

[{'role': 'user',
  'content': '###Instructions### Predict which three of the following future signals are the most relevant to user input. You have to choose three of these signals. \n\n###Future signal summaries###\n{signals}\n\n###User input:\n{user_input}'}]

In [28]:
message = MessageTemplate.load(path_prompt_top_signals)
function = FunctionTemplate.load(func_top_signals)

In [32]:
user_input = "I am a parent"

In [33]:
response = TextGenerator.generate(
        model=selected_model,
        temperature=temperature,
        messages=[message],
        message_kwargs={"signals": signals_descriptions, "user_input": user_input},
        stream=False,
        functions=[function.to_prompt()],
        function_call={"name": "predict_top_signals"},
)

In [34]:
answer = json.loads(response['choices'][0]['message']['function_call']['arguments'])
answer

{'prediction': ['Pronatalism vs pro-family',
  'Hidden Figures',
  'Green neighbours']}

In [35]:
answer['prediction'][0:3]

['Pronatalism vs pro-family', 'Hidden Figures', 'Green neighbours']

## Intent detection

In [137]:
path_func_intent = "data/func_intent.json"
path_prompt_intent = "data/prompt_intent.jsonl"


In [138]:
func_intent = json.loads(open(path_func_intent).read())
func_intent

{'name': 'predict_intent',
 'description': "Predict what is the user's intent",
 'parameters': {'type': 'object',
  'properties': {'prediction': {'type': 'string',
    'enum': ['one_specific_signal', 'more_signals', 'following_up'],
    'description': 'The predicted intent'}},
  'required': ['prediction']}}

In [139]:
prompt_intent = read_jsonl(path_prompt_intent)
prompt_intent

[{'role': 'user',
  'content': 'You are a helpful chatbot talking with the user about the future signals. ###Instructions### Predict the intended action of the user, what the user wishes you to carry out based on the user input\n\n###Possible intents###\n{intents}\n\n###User input:\n{user_input}'}]

In [140]:
message = MessageTemplate.load(path_prompt_intent)
function = FunctionTemplate.load(func_intent)

In [157]:
user_input = "Tell me about the robot chef"

In [158]:
response = TextGenerator.generate(
        model=selected_model,
        temperature=temperature,
        messages=[message],
        message_kwargs={"intents": actions_descriptions, "user_input": user_input},
        stream=False,
        functions=[function.to_prompt()],
        function_call={"name": "predict_intent"},
)

In [159]:
answer = json.loads(response['choices'][0]['message']['function_call']['arguments'])
answer

{'prediction': 'one_specific_signal'}

## Prompt: Impact on the user

In [189]:
path_prompt = "data/02_signal_impact.jsonl"

In [190]:
signal = "robochefs"

In [191]:
message = MessageTemplate.load(path_prompt)

In [192]:
user_input = "I like food"

In [193]:
response = TextGenerator.generate(
        model=selected_model,
        temperature=temperature,
        messages=[message],
        message_kwargs={
            "signal": signals_dict[signal]['full_text'], 
            "user_input": user_input
            },
        stream=False,
)

In [194]:
answer = response['choices'][0]['message']['content']
print(answer)

The future signal about robochefs and automation in commercial kitchens might be relevant to you as someone who likes food because it explores the potential impact of this technology on our diets and the food industry. 

Three ways this future signal might impact you are: 
1. It could lead to the automation of food preparation and distribution, making meals faster and potentially cheaper.
2. There might be a relative reduction in the price of fast food compared to healthier options if innovation is primarily focused on mass production.
3. However, there is also potential for healthier processes and options to be developed, such as using air frying instead of deep frying, and chains specializing in healthier food getting involved in kitchen robotics.

If you have any questions about how this technology might impact the food industry or your own food choices, feel free to ask. Alternatively, you can also ask about other future signals that interest you.


## Prompt: Summary of different signals

In [9]:
path_prompt = "data/03_signal_choice.jsonl"

In [212]:
chosen_signals = ["robochefs", "baby_boom", "abundant_energy"]
signals_text = generate_signals_texts(signals_data, chosen_signals)

In [213]:
message = MessageTemplate.load(path_prompt)

In [219]:
user_input = "I like food"
message_history = [
    MessageTemplate.load({"role": "user", "content": "I'm a parent"}),
    MessageTemplate.load({"role": "user", "content": "I have a heat pump"}),
]

In [236]:
ms = message_history + [message]
ms

[MessageTemplate(initial_template={'role': 'user', 'content': "I'm a parent", 'name': None}, role='user', content="I'm a parent", name=None),
 MessageTemplate(initial_template={'role': 'user', 'content': 'I have a heat pump', 'name': None}, role='user', content='I have a heat pump', name=None),
 MessageTemplate(initial_template={'role': 'user', 'content': 'Start your answer by explaining each of the signals in one clear sentence (use similar language to the signals descriptions). If possible, indicate how a signal might be relevant to the user, given the user information and conversation history. Finish your answer by asking the user to choose one of the signals to hear more about it. Remember that you must be patient and never offend or be aggressive.   \n\n###Future signals###{signals}\n\n###User information### Here is what the user told you about themselves: {user_input}.\n\n###Answer###', 'name': None}, role='user', content="Start your answer by explaining each of the signals in on

In [237]:
response = TextGenerator.generate(
        model=selected_model,
        temperature=temperature,
        messages=ms,
        message_kwargs={
            "signals": signals_text, 
            "user_input": user_input
            },
        stream=False,
)

In [238]:
answer = response['choices'][0]['message']['content']
print(answer)

Given your interest in food, you might find the signal 'robochefs' particularly relevant as it discusses the intersection of technology and culinary practices. Robochefs are a new development where automation is being integrated into commercial kitchens, potentially influencing the way food is prepared and served. Companies are exploring healthier cooking methods and fresh ingredients, but the industry is facing economic challenges with a need for clearer benefits to attract investment.

The signal 'abundant_energy' could be interesting to you as a parent and homeowner with a heat pump, because it talks about the future of energy sources, like solar and wind power, which could affect how you power your home and devices, including heating and cooling systems. The article touches on the possibility of energy becoming more affordable and abundant, which can lead to innovative uses and impact environmental sustainability.

Lastly, the signal 'baby_boom' touches on societal trends and polic